In [50]:
# load libraries
import numpy as np
import pyxpcm
from pyxpcm.models import pcm

import xarray as xr

from argopy import DataFetcher as ArgoDataFetcher

import cartopy.crs as ccrs 
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import gsw 

In [9]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=4000
# January 
time_in='2018-01-01'
time_f='2018-01-03'

In [38]:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    # Function to get argo data for a given lat,lon box (using Argopy), 
    # and return a 2D array collection of vertical profile for the given region
    #llon: left longitude
    #rlon: right longtidue
    #ulat: upper latitude
    #llat: lower latitude

    ds_points = ArgoDataFetcher(src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

In [88]:
def spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end):
    #Function that will get the argo data
    #llon: left longitude
    #rlon: right longtidue
    #ulat: upper latitude
    #llat: lower latitude
    #time
    
    max_dt = timedelta(days = 1)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)
        return ds
    else:
        early_end = time_start+max_dt
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,early_end)
        
        ds2 = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax, early_end,time_end)
        return xr.concat([ds1,ds2],dim='N_PROF')


In [90]:
ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f)
ds

<xarray.Dataset>
Dimensions:          (N_LEVELS: 1125, N_PROF: 51)
Coordinates:
    LATITUDE         (N_PROF) float64 23.64 60.08 32.06 ... 55.29 28.83 60.32
    TIME             (N_PROF) datetime64[ns] 2018-01-02T01:04:00 ... 2018-01-...
    LONGITUDE        (N_PROF) float64 -45.48 -52.81 -55.31 ... -60.68 -26.21
  * N_PROF           (N_PROF) int64 36 22 10 21 19 13 4 ... 48 49 47 42 35 16 32
  * N_LEVELS         (N_LEVELS) int64 0 1 2 3 4 5 ... 1120 1121 1122 1123 1124
Data variables:
    CYCLE_NUMBER     (N_PROF) int64 30 9 125 9 9 293 14 ... 52 101 80 55 33 95
    DATA_MODE        (N_PROF) <U1 'D' 'A' 'D' 'A' 'D' ... 'R' 'D' 'D' 'D' 'A'
    DIRECTION        (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER  (N_PROF) int64 6901198 4902419 4901705 ... 4902354 6901176
    POSITION_QC      (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PRES             (N_PROF, N_LEVELS) float32 4.0 10.3 15.1 ... nan nan nan
    PRES_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PSAL             (N_PROF, N_LEVELS) float32 37.46539 37.46539 ... nan nan
    PSAL_QC          (N_PROF) int64 2 1 1 1 1 2 1 1 1 1 ... 1 2 1 1 1 1 1 1 1 1
    TEMP             (N_PROF, N_LEVELS) float32 25.22 25.219 25.22 ... nan nan
    TEMP_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    TIME_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://www.ifremer.fr/erddap
    Fetched_by:           jovyan
    Fetched_date:         2020/08/14
    Fetched_constraints:  phy_[x=-90.00/0.00; y=0.00/70.00; z=0.0/4000.0; t=2...
    Fetched_uri:          https://www.ifremer.fr/erddap/tabledap/ArgoFloats.n...
    history:              Variables filtered according to DATA_MODE; Variable...

In [39]:
time_in='2018-01-01'
time_f='2018-01-02'
ds1 = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f)

In [40]:
time_in='2018-01-02'
time_f='2018-01-03'
ds2 = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f)

In [41]:
ds_merged = xr.concat([ds1,ds2],dim='N_PROF')

In [42]:
ds_merged

<xarray.Dataset>
Dimensions:          (N_LEVELS: 1156, N_PROF: 123)
Coordinates:
  * N_LEVELS         (N_LEVELS) int64 0 1 2 3 4 5 ... 1151 1152 1153 1154 1155
    LATITUDE         (N_PROF) float64 50.08 63.98 24.82 ... 55.29 28.83 60.32
    TIME             (N_PROF) datetime64[ns] 2018-01-01T00:12:23 ... 2018-01-...
    LONGITUDE        (N_PROF) float64 -12.57 -55.93 -89.41 ... -60.68 -26.21
  * N_PROF           (N_PROF) int64 61 58 17 9 18 16 23 ... 48 49 47 42 35 16 32
Data variables:
    CYCLE_NUMBER     (N_PROF) int64 124 71 217 40 160 188 ... 52 101 80 55 33 95
    DATA_MODE        (N_PROF) <U1 'D' 'D' 'D' 'R' 'D' ... 'R' 'D' 'D' 'D' 'A'
    DIRECTION        (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER  (N_PROF) int64 6901920 6901721 4901479 ... 4902354 6901176
    POSITION_QC      (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PRES             (N_PROF, N_LEVELS) float32 10.6 15.2 20.1 ... nan nan nan
    PRES_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PSAL             (N_PROF, N_LEVELS) float32 35.491 35.492 35.492 ... nan nan
    PSAL_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 2 1 1 1 1 1 1 1 1
    TEMP             (N_PROF, N_LEVELS) float32 12.119 12.119 12.12 ... nan nan
    TEMP_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    TIME_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://www.ifremer.fr/erddap
    Fetched_by:           jovyan
    Fetched_date:         2020/08/13
    Fetched_constraints:  phy_[x=-90.00/0.00; y=0.00/70.00; z=0.0/4000.0; t=2...
    Fetched_uri:          https://www.ifremer.fr/erddap/tabledap/ArgoFloats.n...
    history:              Variables filtered according to DATA_MODE; Variable...

In [63]:
d1 = datetime.strptime(time_in, "%Y-%m-%d")
d2 = datetime.strptime(time_f, "%Y-%m-%d")
max_dt = timedelta(hours = 2.5)
#if d2 - d1 <= max_dt
#else split chunks

In [85]:
def spliced(time_start, time_end):
    
    max_dt = timedelta(days = 10)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        print(time_start, time_end)
    else:
        early_end = time_start+max_dt
        print(time_start, early_end)
        spliced(early_end, time_end)

In [86]:
spliced(time_in, time_f)

2018-01-02 00:00:00 2018-01-03 00:00:00


In [87]:
d3 = datetime(2018,3,2)
spliced(d1,"2018-03-02")

2018-01-02 00:00:00 2018-01-12 00:00:00
2018-01-12 00:00:00 2018-01-22 00:00:00
2018-01-22 00:00:00 2018-02-01 00:00:00
2018-02-01 00:00:00 2018-02-11 00:00:00
2018-02-11 00:00:00 2018-02-21 00:00:00
2018-02-21 00:00:00 2018-03-02 00:00:00


In [58]:
dt = timedelta(days=10)

In [69]:
(d2-d1).total_seconds()/max_dt.total_seconds()

9.6

9000.0

datetime.timedelta(days=1)

In [83]:
if isinstance(time_in, str):
    time_in = datetime.strptime(time_in,"%Y-%m-%d")
if isinstance(time_f, str):
    time_f = datetime.strptime(time_f,"%Y-%m-%d")

True